In [78]:
import pandas as pd
import swifter

In [79]:
vs='/home/ndo/bash_output/04_callvariant_vcf/vsc_mincov2.vcf'
df = pd.read_csv(vs, sep="\t")
df

,Chrom,Position,Ref,Cons,Reads1,Reads2,VarFreq,Strands1,Strands2,Qual1,Qual2,Pvalue,MapQual1,MapQual2,Reads1Plus,Reads1Minus,Reads2Plus,Reads2Minus,VarAllele
0,chr1,601436,C,Y,2,2,50%,1,2,37,37,0.98,1,1,2,0,1,1,T
1,chr1,601667,C,Y,1,2,66.67%,1,2,37,37,0.98,1,1,0,1,1,1,T
2,chr1,604385,C,M,1,1,50%,1,1,37,37,0.98,1,1,1,0,0,1,A
3,chr1,604531,T,Y,2,1,33.33%,1,1,37,37,0.98,1,1,0,2,0,1,C
4,chr1,604726,A,M,1,1,50%,1,1,37,25,0.98,1,1,1,0,0,1,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4903347,chr22,50791067,C,Y,1,1,50%,1,1,37,37,0.98,1,1,1,0,1,0,T
4903348,chr22,50792075,G,K,3,2,40%,2,1,37,31,0.98,1,1,2,1,2,0,T
4903349,chr22,50792295,C,M,7,1,12.5%,2,1,37,25,0.98,1,1,3,4,1,0,A
4903350,chr22,50793095,T,Y,3,1,25%,2,1,37,37,0.98,1,1,1,2,0,1,C


In [80]:
# df.drop_duplicates(subset=['Chrom', 'Position'])

In [81]:
df=df[['Chrom','Position','Ref','VarAllele']]
df = df.rename(columns={'Chrom' : '#CHROM', 'Position' : 'POS', 'Ref' : "REF",'VarAllele':'ALT'}) #'#CHROM','POS','REF','ALT'
df

,#CHROM,POS,REF,ALT
0,chr1,601436,C,T
1,chr1,601667,C,T
2,chr1,604385,C,A
3,chr1,604531,T,C
4,chr1,604726,A,C
...,...,...,...,...
4903347,chr22,50791067,C,T
4903348,chr22,50792075,G,T
4903349,chr22,50792295,C,A
4903350,chr22,50793095,T,C


In [82]:
## DEDUPLICATE VARIANTS BASED ON CHROM AND POS
# df[df.duplicated(subset=["#CHROM", "POS"]) == True]
# df = df[df.duplicated(["#CHROM",'POS'], keep=False)]

In [83]:
def read_vcf_with_header(vcf_file):
    """
    Read VCF file into pandas DataFrame and extract header lines.
    """
    vcf_header = []
    with open(vcf_file, "r") as f:
        for line in f:
            if line.startswith('#CHROM'):
                vcf_header.extend(line.strip().split('\t'))
    df = pd.read_csv(vcf_file, sep="\t", comment="#")
    df.columns = vcf_header
   
    return df

vcf_GIAB= "/home/ndo/GIAB-GT/GIAB_SNP.vcf"
df_giab= read_vcf_with_header(vcf_GIAB)
df_giab = df_giab[['#CHROM','POS','REF','ALT']]

In [84]:
df_giab

,#CHROM,POS,REF,ALT
0,chr1,783175,T,C
1,chr1,784860,T,C
2,chr1,785417,G,A
3,chr1,797392,G,A
4,chr1,798618,C,T
...,...,...,...,...
3358496,chr22,50791190,G,A
3358497,chr22,50792075,G,T
3358498,chr22,50792591,T,C
3358499,chr22,50792792,A,G


In [85]:
merge_df = pd.merge(df_giab, df, how="outer", on=["#CHROM", "POS"])
m_df = merge_df.rename(columns={"#CHROM" : "CHROM", "REF_x" : "REF_GIAB", "ALT_x" : "ALT_GIAB", "REF_y" : "REF_S", "ALT_y" : "ALT_S"})

In [86]:
def categorize_variants(df):
    if (df["ALT_GIAB"] == df["ALT_S"]) and (df["REF_GIAB"] == df["REF_S"]):
        return "TP"
    if (df["ALT_GIAB"] != df["ALT_S"]) and (df["REF_GIAB"] == df["REF_S"]):
        return "FP"
    if  df["ALT_GIAB"] and pd.isna(df["ALT_S"]):
        return "FN"
    if  pd.isna(df["ALT_GIAB"]) and df["ALT_S"]:
        return "FP"
    return "None"

In [87]:
m_df['VAR_CATE'] = m_df.swifter.apply(categorize_variants, axis = 1)


Pandas Apply: 100%|██████████| 6249768/6249768 [01:42<00:00, 60713.12it/s]


In [88]:
m_df['VAR_CATE'].value_counts()

VAR_CATE
FP      2892193
TP      2011035
FN      1346416
None        124
Name: count, dtype: int64

In [89]:
# num_set = set()
# for i in range(1,23):
#     num_set.add(str(i))

# def categorize_chrom_column(df):
#     if df['Chrom'].startswith('chr') and df['Chrom'][3:] in num_set:
#         return True
#     return False

In [90]:

# df['CHROM'] = df.apply(categorize_chrom_column, axis=1)

In [91]:
#extract rows with weird chrom values
# chrom_df = df[df['CHROM'] == False]
# norm_df = df[df['CHROM'] == True]

In [92]:
# chrom_df['Chrom'].value_counts()
# norm_df['Chrom'].value_counts().sum()

In [93]:
# test = df['Chrom'].str.split('_').tolist()

# def get_chrom_pos(df):
#     if df['Chrom'].startswith('chr'):
#         return None
#     return df['Chrom'].split('_')[1]




In [94]:
# chrom_df['Chrom'] = chrom_df.apply(get_chrom_pos, axis= 1)

In [95]:
# norm_df['Chrom'].value_counts().head(29)

In [96]:

# filtered_df = norm_df[norm_df['Chrom'].str.startswith('chr') & norm_df['Chrom'].str[3:].astype(int).between(1, 22)]
#

In [97]:
# df = pd.concat([chrom_df,norm_df], ignore_index=True, sort=False)

In [98]:
# norm_df.sort_values(by=['Chrom'], inplace=True)

In [99]:
#write the norm_df into vcf file
# norm_df.to_csv('/home/ndo/bash_output/variants_df/varscan_proccess', sep='\t', index=False, encoding='utf-8')

In [100]:
#insertect with bam first and do variant calling later